In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from datetime import datetime
from datetime import timedelta, date
import matplotlib.dates as mdates
import math
import warnings

from sklearn import neighbors
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

from sklearn import preprocessing
from sklearn.preprocessing import Imputer

from random import randint

warnings.filterwarnings('ignore')

In [2]:
properties = pd.read_csv('trainTree.csv')
testing = pd.read_csv('testTree.csv')

In [3]:
train = properties
train = train.drop('id', 1)
train = train.drop('description', 1)
train = train.drop('Unnamed: 0', 1)
train = train.drop('created_on', 1)
train['lat'] = train['lat']*2000
train['lon'] = train['lon']*2000

In [4]:
test = testing
test = test.drop('description',1)
test = test.drop('Unnamed: 0',1)
test = test.drop('created_on',1)
test['lat'] = test['lat']*2000
test['lon'] = test['lon']*2000
#t = t.drop('place_name_encoded',1)
#t = t.drop('id',1)

In [5]:
l = len(properties['place_name_encoded'].value_counts())

#CAMBIO: hardcodeo el valor original (place name encoded va hasta 678)


#uso este porque sé que es bajo
trainvacios = train[train['place_name_encoded']==1]
testvacios = test[test['place_name_encoded']==1]

for x in range(0,l):
    if ((len(test[test['place_name_encoded']==x])==0) or (len(train[train['place_name_encoded']==x])<50)) and (x!=1):
    #excluyo el hardcodeado
    
        train0 = train[train['place_name_encoded']==x]
        trainvacios = pd.concat([trainvacios, train0])
        
        test0 = test[test['place_name_encoded']==x]
        testvacios = pd.concat([testvacios, test0])

In [6]:
#los datos restantes tienen más de 10 entradas en train y al menos alguna en test por barrio
#los junto en el dataframe 'trainaptos'

trainaptos= train[~train['place_name_encoded'].isin(trainvacios['place_name_encoded'])]
testaptos= test[~test['place_name_encoded'].isin(testvacios['place_name_encoded'])]
#trainaptos.info()

In [7]:
#Reviso que quedo bien el split

total = pd.concat([trainaptos, trainvacios])
print('Train:')
print('Registros:      ', len(total), ' = ', len(train))
print('Aptos + Vacios: ', len(trainaptos), '+', len(trainvacios))

print('Place_names:    ', len(train['place_name_encoded'].value_counts()), ' = ', len(total['place_name_encoded'].value_counts()))
print('Aptos + Vacios: ', len(trainaptos['place_name_encoded'].value_counts()), '+', len(trainvacios['place_name_encoded'].value_counts()))
print()


total = pd.concat([testaptos, testvacios])
print('Test:')
print('Registros:      ', len(total), ' = ', len(test))
print('Aptos + Vacios: ', len(testaptos), '+', len(testvacios))

print('Place_names:    ', len(test['place_name_encoded'].value_counts()), ' = ', len(total['place_name_encoded'].value_counts()))
print('Aptos + Vacios: ', len(testaptos['place_name_encoded'].value_counts()), '+', len(testvacios['place_name_encoded'].value_counts()))

Train:
Registros:       609670  =  609670
Aptos + Vacios:  422499 + 187171
Place_names:     899  =  899
Aptos + Vacios:  190 + 709

Test:
Registros:       14166  =  14166
Aptos + Vacios:  14107 + 59
Place_names:     223  =  223
Aptos + Vacios:  190 + 33


In [8]:
#armo un dataframe con el cluster, un numero aleatorio, score y tamaño en trainaptos
cant_metodos = 2

#from random import randint
clusters = pd.DataFrame()

for x in range(0,l):
    clusx = {'cluster' : pd.Series([x], index=[x]),
    'trainaptos_size' : pd.Series([len(trainaptos[trainaptos['place_name_encoded']==x])], index=[x]),
    'method' : pd.Series([randint(1, cant_metodos)], index=[x]),
    'score' : pd.Series([0], index=[x]),}
    df = pd.DataFrame(clusx)
    clusters = pd.concat([clusters,df])
clusters.sample(5)

,cluster,method,score,trainaptos_size
756,756,2,0,0
456,456,1,0,0
777,777,1,0,0
197,197,1,0,50
316,316,2,0,0


In [9]:
#armo una funcion que devuelva un dataframe con las predicciones del test y train dados y su id

#NOTA:
#cambie como toma los parametros, y solo devuelve las prediciones como serie
#no como dataframe con formato para entregar
#no hice lo mismo con KNN todavia




def RandomForest(test,train):
    train = train.drop('place_name_encoded',1)
    
    trainLX = train.drop('price_aprox_usd', 1)
    trainLY = train[['price_aprox_usd']]
    
    #t = test.drop('id',1)
    t = test.drop('place_name_encoded',1)
    
    regr = RandomForestRegressor(min_samples_leaf=1, min_samples_split=2,n_estimators=500, n_jobs = -1,max_features='log2')
    predictions = regr.fit(trainLX, trainLY).predict(t)
    predictions = predictions.flatten()

    #ids = test.as_matrix()[:,0].flatten()
    #results = pd.DataFrame()
    #results['id'] = pd.Series(ids.astype('int32'))
    #results['price_usd'] = pd.Series(predictions)
    results=predictions
    
    return results

In [10]:
#armo una funcion que devuelve los resultados de predicciones KNN

def KNN(test,train,n):
    train = train.drop('place_name_encoded',1)
    train_no_price = train.drop('price_aprox_usd', 1)
    trainprice = train.loc[:, 'price_aprox_usd']
    test = test.drop('place_name_encoded',1)
    testnoid = test.drop('id',1)
    
    knn = neighbors.KNeighborsRegressor(n, weights="uniform", algorithm="ball_tree")

    predictions = knn.fit(train_no_price, trainprice).predict(testnoids)
    predictions = predictions.flatten()
    #ids = test.as_matrix()[:,0].flatten()
    #results = pd.DataFrame()
    #results['id'] = pd.Series(ids.astype('int32'))
    #results['price_usd'] = pd.Series(predictions)
    results=predictions
    
    return results

In [24]:
#corro pruebas con el dataset
l = len(properties['place_name_encoded'].value_counts())

#fraccion en la que separo train
tamano_prueba = 0.8

for x in range(0,l):
    if len(testaptos[testaptos['place_name_encoded']==x])!=0:
        trainx = trainaptos[trainaptos['place_name_encoded']==x]
        #separo trainx en trainprueba y test prueba
        trainprueba = trainx.sample(round(len(trainx)*tamano_prueba))
        testprueba = trainx[~trainx.isin(trainprueba)]
        testprueba.dropna(inplace=True)
        
        #tomo valores reales de testprueba, se los saco para hacer la regresion
        real = testprueba['price_aprox_usd'].values
        testprueba= testprueba.drop('price_aprox_usd', 1)
        
        #arranco usando siempre random forest por ahora
        #la idea es usar el metodo que dicta el dataframe clusters en 'metodo' (que es aleatorio)
        ################################
        #FALTA AGREGAR MÁS METODOOOOOOS#
        ################################
        if (clusters['method'][x]==1):
            resx = RandomForest(testprueba,trainprueba)
        else:
            #aca iria KNN por ejemplo
            resx = RandomForest(testprueba,trainprueba)
        
        #anoto el score como el error absoluto medio entre precios reales y prediccion
        #sumo 1 en caso (super copado) que toque 0, no terminaria afectando el orden
        clusters['score'][x] = median_absolute_error(real, resx)+1

In [14]:
#muestro los valores de cluster con score distinto de 0 (los que usa trainaptos)
clusters[cluster['score']!=0]

,cluster,method,score,trainaptos_size
0,0,1,0,0
1,1,1,0,0
2,2,1,0,0
3,3,2,934,868
4,4,1,8936,199
5,5,1,0,0
6,6,2,0,0
7,7,2,1619,2885
8,8,2,0,0
9,9,1,0,0


In [69]:
c = clusters.sort_values('score', ascending=False)
c = c[c['score']!=0]
peor = c.head(round(len(c)*5/100)).sample()
peor

,cluster,method,score,trainaptos_size
14,14,1,8781,63


In [70]:
print(peor.iloc[0]['method'])

1


In [86]:
#devuelve numero aleatorio entre 1 y cant_metodos, que no sea el que se encuentra en method
random.choice(list(range(1,peor.iloc[0]['method'])) + list(range(peor.iloc[0]['method']+1,cant_metodos+1)))

2

In [14]:
#ordeno clusters segun el peor median error y quito los valores de vacios
#ahora tomo una muestra aleatoria del 10% del mismo, y pruebo si cambiando a otro metodo mejora

n_pruebas=50
porcentaje=5

for x in (0,n_pruebas):
    #ordeno clusters segun el peor median error y quito los valores de vacios
    #ahora tomo una muestra aleatoria del 10% del mismo, y pruebo si cambiando a otro metodo mejora
    c = clusters.sort_values('score', ascending=False)
    c = c[c['score']!=0]
    peor = c.head(round(len(c)*porcentaje/100))
    #peor es una fila random del 5% de peor puntaje
    
    #escojo un nuevo metodo: un aleatorio entre 1  y cantidad de metodos que NO sea el que acabo de usar
    nuevo_metodo = random.choice(list(range(1,peor.iloc[0]['method'])) + list(range(peor.iloc[0]['method']+1,cant_metodos+1)))
    
    trainx = trainaptos[trainaptos['place_name_encoded']==peor.iloc[0]['cluster']]
    #separo trainx en trainprueba y test prueba
    trainprueba = trainx.sample(round(len(trainx)*tamano_prueba))
    testprueba = trainx[~trainx.isin(trainprueba)]
    testprueba.dropna(inplace=True)
    
    #tomo valores reales de testprueba, se los saco para hacer la regresion
    real = testprueba['price_aprox_usd'].values
    testprueba= testprueba.drop('price_aprox_usd', 1)
    
    #arranco usando siempre random forest por ahora
    #mismo que antes, uso el metodo que indica ahora nuevo metodo
    
    ################################
    #FALTA AGREGAR MÁS METODOOOOOOS#
    ################################
    if (nuevometodo==1):
        resx = RandomForest(testprueba,trainprueba)
    else:
        #aca iria KNN por ejemplo
        resx = RandomForest(testprueba,trainprueba)
    
    nuevo_score = median_absolute_error(real, resx)+1
    
    #si el nuevo score supera el score original, reemplazo el metodo y score en el df por los nuevos
    if(nuevo_score>peor.iloc[0]['score']):
        clusters['score'][peor.iloc[0]['cluster']] = nuevo_score
        clusters['method'][peor.iloc[0]['cluster']] = nuevo_metodo

,cluster,method,score,trainaptos_size
420,420,1,82151,79
797,797,2,22323,183
212,212,2,19502,232
17,17,2,16162,357
229,229,2,15748,3756
421,421,1,13924,58
716,716,2,11108,75
216,216,1,10722,303
838,838,2,8337,440
306,306,2,7727,250


In [ ]:
#AHORA, ya tenemos cluster cargado con el mejor metodo para cada placename o cluster
#(bah, el que dio mejor resultado al menos)
#Ya puedo hacer la regresion posta

In [ ]:
#arranco con test/trainvacios
#uso random forest por ahora pero podemos hacer lo mismo que con los de arriba mas tarde

predicciones = RandomForest(testvacios,trainvacios)
#ahora que cambie la funcion random forest (para que saque solo las predicciones, porque facilita el calculo del score)
#tengo que pasarlo a dataframe afuera de la funcion
resultados = pd.DataFrame()
resultados['id'] = pd.Series(ids.astype('int32'))
resultados['price_usd'] = pd.Series(predicciones)

resultados.info()

In [28]:
l = len(properties['place_name_encoded'].value_counts())

for x in range(0,l+2):
    if len(testaptos[testaptos['place_name_encoded']==x])!=0:
        
        testx = testaptos[testaptos['place_name_encoded']==x]
        trainx = trainaptos[trainaptos['place_name_encoded']==x]
        
        #MISMO, ACA PONGO LOS METODOS QUE FALTAN
        
        metodo = clusters['method'][x]
        if (metodo==1):
            predicciones = RandomForest(testx,trainx)
        else:
            predicciones = RandomForest(testx,trainx)
        
        #paso las predicciones a dataframe para entregar
        ids = testx.as_matrix()[:,0].flatten()
        resx = pd.DataFrame()
        resx['id'] = pd.Series(ids.astype('int32'))
        resx['price_usd'] = pd.Series(predicciones)
        
        resultados = pd.concat([resultados, resx])

17852
4463
22315


In [ ]:
#debe contener la misma cantidad que testing (14166)
resultados.info()

In [ ]:
resultados.set_index(['id'], inplace=True)
resultados.to_csv('predicciones_RF_placename.csv')